In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('trairt').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
all_data = spark.read.csv('/train.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
all_data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['timestamp', 'Asset_ID', 'Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'Target']

In [6]:
all_data.head(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(timestamp=1514764860, Asset_ID=2, Count=40.0, Open=2376.58, High=2399.5, Low=2357.14, Close=2374.59, Volume=19.23300519, VWAP='2373.1163915061647', Target=-0.004218152387429286), Row(timestamp=1514764860, Asset_ID=0, Count=5.0, Open=8.53, High=8.53, Low=8.53, Close=8.53, Volume=78.38, VWAP='8.53', Target=-0.014398966468964769)]

In [7]:
all_data.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-----------------+-----------------+--------------------+--------------------+------------------+------------------+-------------------+-------------------+--------------------+
|summary|           timestamp|         Asset_ID|            Count|                Open|                High|               Low|             Close|             Volume|               VWAP|              Target|
+-------+--------------------+-----------------+-----------------+--------------------+--------------------+------------------+------------------+-------------------+-------------------+--------------------+
|  count|            24236806|         24236806|         24236806|            24236806|            24236806|          24236806|          24236806|           24236806|           24236797|            23486468|
|   mean|1.5771204253124714E9| 6.29254353894651|286.4593481500822|   1432.640491617432|  1436.3503906875278|1429.5675986148337|1432.6399774617075|  286852.9856098686| 1

In [12]:
#timestamp: All timestamps are returned as second Unix timestamps (the number of seconds elapsed since 1970-01-01 00:00:00.000 UTC). Timestamps in this dataset are multiple of 60, indicating minute-by-minute data.
#Asset_ID: The asset ID corresponding to one of the crytocurrencies (e.g. Asset_ID = 1 for Bitcoin). The mapping from Asset_ID to crypto asset is contained in asset_details.csv.
#Count: Total number of trades in the time interval (last minute).
#Open: Opening price of the time interval (in USD).
#High: Highest price reached during time interval (in USD).
#Low: Lowest price reached during time interval (in USD).
#Close: Closing price of the time interval (in USD).
#Volume: Quantity of asset bought or sold, displayed in base currency USD.
#VWAP: The average price of the asset over the time interval, weighted by volume. VWAP is an aggregated form of trade data.
#Target: Residual log-returns for the asset over a 15 minute horizon.

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
all_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- timestamp: integer (nullable = true)
 |-- Asset_ID: integer (nullable = true)
 |-- Count: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- VWAP: string (nullable = true)
 |-- Target: double (nullable = true)

In [9]:
all_data.createOrReplaceTempView('kick')
c = spark.sql("SELECT COUNT(*) FROM kick")
c.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|24236806|
+--------+

In [27]:
df_new.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- timestamp: integer (nullable = true)
 |-- Asset_ID: integer (nullable = true)
 |-- Count: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- VWAP: double (nullable = true)
 |-- Target: double (nullable = true)

In [10]:
from pyspark.sql.types import DoubleType
df_new = all_data.withColumn("VWAP", all_data["VWAP"].cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_newer = df_new.na.drop(how='any')
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Asset_ID', 'Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP'], outputCol='features')
output = assembler.transform(df_newer)
final_data = output.select('features','Target')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_newer.createOrReplaceTempView('kick2')
d = spark.sql("SELECT COUNT(*) FROM kick2")
d.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|23486459|
+--------+

In [13]:
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
train, test = final_data.randomSplit([0.7, 0.3])
rfc = RandomForestRegressor(numTrees=1000, labelCol='Target')
gbt = GBTRegressor(labelCol='Target', maxMemoryInMB=512, maxIter=100, seed=2)
#rfc_fit = rfc.fit(train)
#rfc_preds = rfc_fit.transform(test)
gbt_fit = gbt.fit(train)
gbt_preds = gbt_fit.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator
my_eval2 = RegressionEvaluator(labelCol='Target')
my_eval2.evaluate(gbt_preds)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.005650909948968013

In [15]:
#gbt_preds.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
from pyspark.ml.feature import Normalizer

normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)
l1NormData = normalizer.transform(final_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
normaltrain, normaltest = l1NormData.randomSplit([0.7, 0.3])
normalrfc = RandomForestRegressor(numTrees=1000, labelCol='Target', maxMemoryInMB=512)
#gbt = GBTRegressor(labelCol='Target', maxMemoryInMB=512, maxIter=100, seed=2)
normalrfc_fit = normalrfc.fit(normaltrain)
normalrfc_preds = normalrfc_fit.transform(normaltest)
#gbt_fit = gbt.fit(train)
#gbt_preds = gbt_fit.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…